### Create Character Vector Database

In [3]:
# Import necessary modules
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import CohereEmbeddings
import json
import random
from langchain.vectorstores import Chroma

with open('config.json', 'r') as f:
    config = json.load(f)
    game_name = config['game_name']
    character = config['characters'][0]
    character_name = character['name']

# Load the character's knowledge file
with open(f'{game_name}/characters/{character_name}/character_knowledge.txt', errors='ignore') as f:
    public_info = f.read()

# Create a text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=0,
    length_function=len,
    separators=['\n\n\n', '\n\n', '\n', '.', ',', ' ']
)

# Split the text
public_info_list = text_splitter.split_text(public_info)

# Randomly select an API key
selected_key = json.load(open('apikeys.json', 'r'))['api_keys'][random.randint(
    0, len(json.load(open('apikeys.json', 'r'))['api_keys'])-1)]

# Initialize the embedding model
embeddings = CohereEmbeddings(cohere_api_key=selected_key, user_agent='CharacterProfileGenerator/1.0')

# Set the persist directory
persist_directory = f'{game_name}/characters/{character_name}/vectordb'

# Create and persist the vector database
vectordb = Chroma.from_texts(public_info_list, embeddings, persist_directory=persist_directory)
vectordb.persist()
